## Biostat/Biomath M257 Homework 6
###### 'Due June 9 @ 11:59PM'
#### author: Sisi Shao (205645669)

System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 10 × Apple M1 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 1 on 8 virtual cores


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Downloads/SSS/Courses/Bio257_23Spring/HW/hw6`


Status `~/Downloads/SSS/Courses/Bio257_23Spring/HW/hw6/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [336ed68f] CSV v0.10.11
  [a93c6f00] DataFrames v1.5.0
  [31c24e10] Distributions v0.25.95
  [b6b21f68] Ipopt v1.4.1
  [67920dd8] KNITRO v0.13.2
  [b8f27783] MathOptInterface v1.17.1
  [ff71e718] MixedModels v4.14.1
  [76087f3c] NLopt v0.6.5
  [08abe8d2] PrettyTables v2.2.4
  [6f49c342] RCall v0.13.15
  [8bb1440f] DelimitedFiles
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


In this assignment, we continue with the linear mixed effects model (LMM) considered in HW3
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T.
$$
Because the variance component parameter $\boldsymbol{\Sigma}$ has to be positive semidefinite. We prefer to use its Cholesky factor $\mathbf{L}$ as optimization variable. 

Given $m$ independent data tuples $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2).
$$
In this assignment, we use the nonlinear programming (NLP) approach for optimization. In HW7, we will derive an EM (expectation-maximization) algorithm for the same problem. There is also an MM (minorization-maximization) algorithm for the same problem; see [this article](https://doi.org/10.1080/10618600.2018.1529601).

In [3]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathOptInterface, MixedModels, NLopt
using PrettyTables, Random, RCall

const MOI = MathOptInterface

[ Info: Precompiling MixedModels [ff71e718-51f3-5ec2-a782-8ffcbfa3c316]


MathOptInterface

## Q1. (Optional, 30 bonus pts) Derivatives

NLP optimization solvers expect users to provide at least a function for evaluating objective value. If users can provide further information such as gradient and Hessian, the NLP solvers will be more stable and converge faster. Automatic differentiation tools are becoming more powerful but cannot apply to all problems yet.

1. Show that the gradient of $\ell_i$ is
\begin{eqnarray*}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i, \\
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i, \\
\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L},
\end{eqnarray*}
where $\mathbf{r}_i = \mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}$. 

2. Derive the observed information matrix and the expected (Fisher) information matrix.

If you need a refresher on multivariate calculus, my [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html) may be helpful.


## Sol Q1

### Sol Q1.1

**Step 1**: We cite several results from [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html).

- (216) $D \phi(\mathbf{x})=\nabla \phi(\mathbf{x})^T$; $D \phi(\mathbf{X})=\left(\text{ vec }\frac{\partial\phi(\mathbf{x})}{\partial\mathbf{X}}\right)^T$ and the chain rule
$$D_{\mathbf{X}}\mathbf{Z}=D_\mathbf{Y}\mathbf{Z}\cdot D_\mathbf{X}\mathbf{Y}$$
- (216) For a scalr-valued matrix function $\phi(\mathbf{X})$, we have
$$D𝜙(𝐗)=(vec\frac{∂𝜙(𝐗)}{∂𝐗})′$$
- (216) Let $A(\alpha)$ be a matrix and its entries are functions of a scalar $\alpha$, then:
\begin{align}
\frac{d}{d\alpha}A^{-1}(\alpha)&=-A^{-1}(\alpha)\frac{dA(\alpha)}{d\alpha}A^{-1}(\alpha)\\
\frac{d}{d\alpha}\log\det A(\alpha)&=\text{Tr}\left(A^{-1}(\alpha)\frac{dA(\alpha)}{d\alpha}\right)
\end{align}
- (216) If $\phi(\mathbf{X})=\text{Tr}(\mathbf{AX}^{-1})$, then
$$\frac{\partial}{\partial\mathbf{X}}\phi(\mathbf{X})=-(\mathbf{X}^{-1}\mathbf{A}\mathbf{X}^{-1})^T$$

- (216) If $\phi(\mathbf{x})=\mathbf{x}^T\mathbf{Ax}$, then
$$\frac{\partial}{\partial\mathbf{X}}\phi(\mathbf{X})=\mathbf{x}^T(\mathbf{A}+\mathbf{A}^T)$$

**Step 2** $\nabla_\beta$ and $\nabla_{\sigma^2}$: Let 
$$f(\sigma^2,\mathbf{L},\beta)=(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})=\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i$$ and $$g(\sigma^2,\mathbf{L})=\log\det(\mathbf{\Omega}_i)$$ then by the chain rule and other results from 216 and 279:
$$\nabla_\beta f=-2\mathbf{X}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i$$
For $\sigma^2$, re-write $\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i=\text{Tr}\left(\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\right)$. Then I need to apply the first result:
\begin{align}
\nabla_{\sigma^2} f&=-\left(\text{vec }(\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1})\right)^T(\text{vec }\mathbf{I}_{n_i})\\
&=-\text{Tr}\left[\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1})\right]\\
&=-\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{r}_i
\end{align}
Similarly,
\begin{align}
\nabla_{\sigma^2} g&=\text{Tr}(\boldsymbol{\Omega}_i^{-1}\mathbf{I}_{n_i})=\text{Tr}(\boldsymbol{\Omega}_i^{-1})
\end{align}
In conclusion,
\begin{align}
\nabla_{\sigma^2} \ell_i&=-\frac{1}{2}\nabla_{\sigma^2} g-\frac{1}{2}\nabla_{\sigma^2} f\\
&=- \frac{1}{2} \operatorname{Tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i\\
\nabla_{\beta} \ell_i&=-\frac{1}{2}\nabla_{\beta} f\\
&=\mathbf{X}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i
\end{align}

### Step 3 
**Part (a)** $\nabla_L \log\det\mathbf{\Omega}^{-1}_i$: For this one, I need an additional result from 216: 

If $𝐹(𝐗)=𝐀𝐺(𝐗)𝐁𝐻(𝐗)𝐂$, then
$$D𝐹(𝐗)=(𝐂′𝐻(𝐗)′𝐁′⊗𝐀)D𝐺(𝐗)+(𝐂′⊗𝐀𝐺(𝐗)𝐁)D𝐻(𝐗).$$

and another result from [this website](https://ccrma.stanford.edu/~dattorro/matrixcalc.pdf):
$$∇_X a^T\mathbf{XX}^Tb = (ab^T + ba^T)\mathbf{X}$$

Plug-in $\mathbf{X}=\mathbf{L},\mathbf{A}=\mathbf{Z}_i, \mathbf{B}=\mathbf{I}_{p}$ and $\mathbf{C}=\mathbf{Z}_i^T$ and let 
$$G(\mathbf{\mathbf{L}})=L,\ H(\mathbf{\mathbf{L}})=L^T$$

Then we have
$$\frac{\partial \mathbf{\Omega}_i}{\partial \mathbf{L}}=\mathbf{Z}_i\mathbf{L}\otimes \mathbf{Z}_i+\left(\mathbf{Z}_i\otimes\mathbf{Z}_i\mathbf{L}\right)\mathbf{K}$$

where $\mathbf{K}$ is the $qq\times qq$ comutation matrix.

Next, by **Roth column lemma**,

$$\left(\text{vec}\frac{\partial \log\det\mathbf{\Omega}_i^{-1}}{\partial\mathbf{\Omega}_i}\right)^T\frac{\partial \mathbf{\Omega}_i}{\partial \mathbf{L}}=2(\text{vec }\mathbf{L}^T\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i)^T$$

Re-shape it into matrix form:
$$\frac{\partial g(\sigma^2,\mathbf{L})}{\partial\mathbf{L}}=2\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L}$$

**Part (b)** $\nabla_Lr_i^T\mathbf{\Omega}_i^{-1}r_i$: For this one, all I need are the ***chain rule*** and the following formula.

- (216) If $\phi(\mathbf{X})=\text{Tr}(\mathbf{AX}^{-1})$, then
$$\frac{\partial}{\partial \mathbf{X}}\phi(\mathbf{X})=-(\mathbf{X}^{-1}\mathbf{A}\mathbf{X}^{-1})^T$$

Note that $$\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i=\text{Tr}(\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1})$$ 
Thus,
$$\frac{\partial}{\partial \mathbf{\Omega}_i}\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i=-\mathbf{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}$$
Hence, again by **Roth column lemma**
$$\left(\text{vec }\frac{\partial}{\partial \mathbf{\Omega}_i}\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i\right)^T\frac{\partial \mathbf{\Omega}_i}{\partial \mathbf{L}}=-2(\text{vec }\mathbf{L}^T\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i)^T$$
Put it into matrix form:
$$\frac{\partial \mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i}{\partial\mathbf{L}}=-2\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L}$$
**Part (3)** $\nabla_L \ell_i$: Combine part (a) and part (b), I get
\begin{align}
\nabla_\mathbf{L} \ell_i&=-\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L}+\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L}
\end{align}


## Q2. (20 pts) Objective and gradient evaluator for a single datum

We expand the code from HW3 to evaluate both objective and gradient. I provide my code for HW3 below as a starting point. You do _not_ have to use this code. If your come up faster code, that's even better. 

## Sol Q2

### Supplementary materials

- $\nabla_{\beta} \ell_i$: By SWM formula in HW2, re-write (drop all unnecessary sub-indexes):
\begin{align}
\mathbf{X}^T\mathbf{\Omega}^{-1}\mathbf{r}&=\frac{1}{\sigma^2}\left((X^Ty-X^TX\beta)-\underbrace{X^TZL}_{(1)}\underbrace{U^{-1}U^{-T}L^TZ^Tr}_{(2)}\right)
\end{align}

where
\begin{align}
U&=\text{chol}(σ²I + L^TZ^TZL)\\
(1)&=\text{transpose(ztx)}L\\
(2)&=U\ \text{\\ storage_q}
\end{align}

- $\nabla_{\sigma^2} \ell_i$: The key is to compute the 2 quantities.

Again by SWM formula,
\begin{align}
\text{Tr}(\Omega^{-1})&=\frac{n}{\sigma^2}-\frac{1}{\sigma^2}\text{Tr}\left(\underbrace{L^TZ^T}_{A^T}ZL(\sigma^2 I_q+L^TZ^TZL)^{-1}\right)\\
&=\frac{n}{\sigma^2}-\frac{1}{\sigma^2}\text{Tr}\left(I_q+\sigma^2(A^TA)^{-1}\right)^{-1}
\end{align}
Next, by SWM formula again,
\begin{align}
\left(I_q+\sigma^2(A^TA)^{-1}\right)^{-1}&=I_q-\left(\frac{1}{\sigma^2} (A^TA) + I_q\right)^{-1}
\end{align}
I get

$$\text{Tr}(\Omega^{-1})=\frac{n-q}{\sigma^2}-\text{Tr}(\sigma^2I_q+A^TA)^{-1}$$

In [4]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T}
    storage_q3 :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2:: Matrix{T}
    storage_Iq :: Matrix{T} 
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2 = Vector{T}(undef, q)
    storage_q3 = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    storage_qq2= similar(ztz)
    storage_Iq = 1.0 * Matrix{T}(I, q, q) 
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, 
        yty, xty, zty, storage_p, storage_q, storage_q2, storage_q3,
        xtx, ztx, ztz, storage_qq, storage_qq2, storage_Iq)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # O(q^3)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # O(pq)
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)    # O(q^2)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # O(q^3)
    # l2 norm of residual vector
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        # TODO: fill ∇β, ∇L, ∇σ² by gradients
        # sleep(1e-3) # pretend this step takes 1ms
        
        # STEP 1:
        # ∇β = X inv(Ω) r
        ## X^TZLU^{-1}storage_q / σ²
        copy!(obs.storage_q2, obs.storage_q)
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q2)
        copy!(obs.storage_q3, obs.storage_q2)
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q2)
        BLAS.gemv!('T', -1/σ², obs.ztx, obs.storage_q2, 0.0, obs.∇β)
        ## (X^Ty-X^TXβ) / σ²
        BLAS.gemv!('N', -1/σ², obs.xtx, β, 1/σ², copy!(obs.storage_p, obs.xty))
        ## Put them together
        axpy!(1.0, obs.storage_p, obs.∇β)
        
        # STEP 2:
        # ∇σ² = -1/2 * Tr(inv(Ω)) + 1/2 * r^T (inv(Ω))^2 r
        ## (q-n)/σ² + Tr(inv(σ²I+A^TA))/σ²
        copy!(obs.storage_qq2, obs.storage_Iq)
        BLAS.trsm!('L', 'U', 'T', 'N', 1.0, obs.storage_qq, obs.storage_qq2)
        obs.∇σ²[1] = (-n + q) / σ² - sum(abs2, obs.storage_qq2)
        ## r^T (inv(Ω))^2 r
        obs.∇σ²[1] += obs.yty / (σ²)^2
        copy!(obs.storage_p, obs.xty)
        obs.∇σ²[1] += dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p)) / (σ²)^2
        obs.∇σ²[1] -= sum(abs2, obs.storage_q3)/σ²
        obs.∇σ²[1] -= sum(abs2, obs.storage_q)/(σ²)^2
        obs.∇σ²[1] /= (2.0)
        
        # STEP 3:
        # ∇L = −𝐙^Tinv(𝛀)𝐙+𝐙^Tinv(𝛀)𝐫𝐫^Tinv(𝛀)𝐙
        ## −𝐙^Tinv(𝛀)𝐙
        copy!(obs.storage_qq2, obs.ztz)
        BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq2)
        BLAS.trsm!('L', 'U', 'T', 'N', 1.0, obs.storage_qq, obs.storage_qq2)
        mul!(obs.∇Σ, transpose(obs.storage_qq2), obs.storage_qq2)
        #rmul!(obs.∇Σ, LowerTriangular(L))
        mul!(obs.∇Σ, obs.ztz, I, T(1), T(-1))
        rdiv!(obs.∇Σ, -σ²)
        ## 𝐙^Tinv(𝛀)𝐫𝐫^Tinv(𝛀)𝐙
        ### 𝐙^Tinv(𝛀)𝐫
        mul!(obs.storage_q, obs.ztz, obs.storage_q2)
        mul!(obs.storage_q, obs.ztx, β, -1.0, -1.0)
        axpy!(1.0, obs.zty, obs.storage_q)
        rdiv!(obs.storage_q, σ²)
        ### Rank 1 update
        BLAS.ger!(1.0, obs.storage_q, obs.storage_q, obs.∇Σ)
    end    
    ###################
    # Return
    ###################        
    return logl    
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in [HW3](https://ucla-biostat-257.github.io/2023spring/hw/hw3/hw03.html).

In [5]:
Random.seed!(257)

# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [6]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇Σ;

logl = logl!(obs, β, L, σ², true) = -3256.179335805838
obs.∇β = [0.26698108057016157, 41.614183370673814, -34.34664962312672, 36.10898510707504, 27.913948208793883]
obs.∇σ² = [1.628371513849288]
obs.∇Σ = [-0.9464482950705874 0.057792444809479926 -0.3024412763913972; 0.057792444809479926 -1.0008716491711904 0.28451165571466047; -0.3024412763913972 0.28451165571466047 1.1700409272597256]


You will lose all 20 points if following statement throws `AssertionError`.

In [7]:
@assert abs(logl - (-3256.1793358058258)) < 1e-4
@assert norm(obs.∇β - [0.26698108057144054, 41.61418337067327, 
        -34.34664962312689, 36.10898510707527, 27.913948208793144]) < 1e-4
# @assert norm(obs.∇Σ - 
#     [-0.9464482950697888 0.057792444809492895 -0.30244127639188767; 
#         0.057792444809492895 -1.00087164917123 0.2845116557144694; 
#         -0.30244127639188767 0.2845116557144694 1.170040927259726]) < 1e-4
@assert abs(obs.∇σ²[1] - (1.6283715138412163)) < 1e-4

### Efficiency

Benchmark for evaluating objective only. This is what we did in HW3.

In [8]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 10000 samples with 172 evaluations.
 Range (min … max):  609.012 ns … 835.756 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     636.140 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   637.109 ns ±  12.925 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

              ▁ ▃▃▄▄▆▇▆▄█▇▇█▄▅▆▅▃▄▂                              
  ▁▁▁▁▂▂▃▃▃▅▆▇█████████████████████▇▅▄▃▃▃▃▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▄
  609 ns           Histogram: frequency by time          680 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Benchmark for objective + gradient evaluation.

In [9]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.583 μs …  3.817 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.629 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.630 μs ± 50.510 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                       ▁  ▄  ▇  █  █  ▆  ▄  ▁                 
  ▂▁▁▂▁▁▂▁▁▃▁▁▃▁▁▅▁▁▇▁▁█▁▁█▁▁█▁▁█▁▁█▁▁█▁▁█▁▁█▁▁▇▁▁▅▁▁▄▁▁▃▁▁▂ ▃
  1.58 μs        Histogram: frequency by time        1.66 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median runt time is 900ns. You will get full credit (10 pts) if the median run time is within 10μs.

In [10]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

## Q3. LmmModel type

We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a `LmmModel` object is simply the sum of log-likelihood/gradient of individual data points. 

In [11]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MOI.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty  :: Vector{T}
    ztr  :: Vector{T}
    ztr2 :: Vector{T}
    xtx  :: Matrix{T}
    ztz  :: Matrix{T}
    ztz2 :: Matrix{T}
    Σ    :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr  = Vector{T}(undef, q)
    ztr2 = Vector{T}(undef, abs2(q))
    xtx  = Matrix{T}(undef, p, p)
    ztz  = Matrix{T}(undef, q, q)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    Σ    = Matrix{T}(undef, q, q)
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr, ztr2, xtx, ztz, ztz2, Σ)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad==true`, then `m.∇β`, `m.∇Σ`, and `m.σ² are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β , 0)
        fill!(m.∇L , 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            BLAS.axpy!(T(1), obs.∇Σ, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    # obtain gradient wrt L: m.∇L = m.∇L * L
    if needgrad
       # TODO 
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), m.L, m.∇L)
    end
    logl
end

logl!

## Q4. (20 pts) Test data

Let's generate a synthetic longitudinal data set to test our algorithm.

In [12]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5; zeros(p - 5)]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0; zeros(q - 3)]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue), Val(true), check=false).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

For later comparison with other software, we save the data into a text file `lmm_data.csv`. **Do not put this file in Git.** It takes 245.4MB storage.

In [13]:
(isfile("lmm_data.csv") && filesize("lmm_data.csv") == 245369685) || 
open("lmm_data.csv", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p-1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q-1)
        print(io, "Z" * string(j) * (j < q-1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:q-1
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end

true

### Correctness

Evaluate log-likelihood and gradient of whole data set at the true parameter values.

In [14]:
copy!(lmm.β, βtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj = logl!(lmm, true)
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L;

obj = logl!(lmm, true) = -2.8400684383699736e6
lmm.∇β = [41.06591670741458, 445.75120353967236, 157.01339922458385, -335.09977360732586, -895.6257448385876]
lmm.∇σ² = [-489.5361730372508]
lmm.∇L = [-3.398257593541934 31.32103842084881 26.736450897360722; 40.43528672995671 61.86377650462431 -75.37427770754628; 37.81105146876985 -82.56838431216376 -56.4599254275938]


Test correctness. You will loss all 20 points if following code throws `AssertError`.

In [15]:
@assert abs(obj - (-2.840068438369969e6)) < 1e-4
@assert norm(lmm.∇β - [41.0659167074073, 445.75120353972426, 
        157.0133992249258, -335.09977360733626, -895.6257448385899]) < 1e-4
@assert norm(lmm.∇L - [-3.3982575935824837 31.32103842086001 26.73645089732865; 
        40.43528672997116 61.86377650461202 -75.37427770754684; 
        37.811051468724486 -82.56838431216435 -56.45992542754974]) < 1e-4
@assert abs(lmm.∇σ²[1] - (-489.5361730382465)) < 1e-4

### Efficiency

Test efficiency.

In [16]:
bm_model = @benchmark logl!($lmm, true)

BenchmarkTools.Trial: 3043 samples with 1 evaluation.
 Range (min … max):  1.596 ms …  1.890 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.640 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.642 ms ± 26.558 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▂▄▆▅▅ ▁▁▁▃▃▃▇██▅▂▁▂▄▂▁▁                                
  ▂▂▃▄▇████████████████████████▆▅▅▃▄▄▃▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▁▂▂ ▄
  1.6 ms         Histogram: frequency by time        1.74 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median run time is 1.4ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [17]:
clamp(10 / (median(bm_model).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [18]:
clamp(10 - median(bm_model).memory / 100, 0, 10)

10.0

## Q5. (30 pts) Starting point

For numerical optimization, a good starting point is critical. Let's start $\boldsymbol{\beta}$ and $\sigma^2$ from the least sqaures solutions (ignoring intra-individual correlations)
\begin{eqnarray*}
\boldsymbol{\beta}^{(0)} &=& \left(\sum_i \mathbf{X}_i^T \mathbf{X}_i\right)^{-1} \left(\sum_i \mathbf{X}_i^T \mathbf{y}_i\right) \\
\sigma^{2(0)} &=& \frac{\sum_i \|\mathbf{r}_i^{(0)}\|_2^2}{\sum_i n_i} = \frac{\sum_i \|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}^{(0)}\|_2^2}{\sum_i n_i}.
\end{eqnarray*}
To get a reasonable starting point for $\boldsymbol{\Sigma} = \mathbf{L} \mathbf{L}^T$, we can minimize the least squares criterion (ignoring the noise variance component)
$$
    \text{minimize} \sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2.
$$
Derive the minimizer $\boldsymbol{\Sigma}^{(0)}$ (10 pts). 

We implement this start point strategy in the function `init_ls()`.

## Sol Q5

We have
\begin{align}
\lVert \mathbf{r}\mathbf{r}^T-\mathbf{Z\Sigma Z}^T \lVert^2_F&=\text{Tr}\left[(\mathbf{r}\mathbf{r}^T-\mathbf{Z\Sigma Z}^T)^T(\mathbf{r}\mathbf{r}^T-\mathbf{Z\Sigma Z}^T)\right]\\
&=\underbrace{\text{Tr}(\mathbf{rr}^T\mathbf{rr}^T)}_{(1)} - \underbrace{\text{Tr}(2\mathbf{Z}^T\mathbf{rr}^T\mathbf{Z\Sigma })}_{(2)}+\underbrace{\text{Tr}(\mathbf{Z\Sigma Z}^T\mathbf{Z\Sigma Z}^T)}_{(3)}
\end{align}
Then 

\begin{align}
\frac{\partial (1)}{\partial\mathbf{\Sigma}}&=\mathbf{O}\\
\frac{\partial (2)}{\partial\mathbf{\Sigma}}&=2\mathbf{Z}^T\mathbf{rr}^T\mathbf{Z} \\
\frac{\partial (3)}{\partial\mathbf{\Sigma}}&=2\mathbf{Z}^T\mathbf{Z\Sigma Z}^T\mathbf{Z}
\end{align}

Set $\frac{\partial (1)}{\partial\mathbf{\Sigma}}-\frac{\partial (2)}{\partial\mathbf{\Sigma}}+\frac{\partial (3)}{\partial\mathbf{\Sigma}}=0$, and then sum over $i=1,2,\cdots,n$:

$$\sum_{i=1}^n\mathbf{Z_i}^T\mathbf{r}_i\mathbf{r}_i^T\mathbf{Z}_i=\sum_{i=1}^n\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{\Sigma}\mathbf{Z}_i^T\mathbf{Z}_i$$

Next, by **Roth column lemma** in [Easy Linear Systems](https://ucla-biostat-257-2021spring.github.io/slides/19-easylineq/easylineq.html),
$$\text{vec} \mathbf{ABC}=(\mathbf{C}^T\otimes\mathbf{A})\text{vec}\mathbf{B}$$
Thus,

$$\text{vec}\left(\sum_{i=1}^n\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{\Sigma}\mathbf{Z}_i^T\mathbf{Z}_i\right)
=\sum_{i=1}^n\left(\mathbf{Z}_i^T\mathbf{Z}_i\otimes\mathbf{Z}_i^T\mathbf{Z}_i\right)\text{vec}\mathbf{\Sigma}=\sum_{i=1}^n\left(\mathbf{Z}_i^T\mathbf{r}_i\otimes \mathbf{Z}_i^T\mathbf{r}_i\right)$$

Hence, the *initial value of $\Sigma$* is

\begin{align}
\text{vec}\mathbf{\Sigma}^{(0)}=\left(\sum_{i=1}^n\mathbf{Z}_i^T\mathbf{Z}_i\otimes\mathbf{Z}_i^T\mathbf{Z}_i\right)^{-1}\left(\sum_{i=1}^n\mathbf{Z}_i^T\mathbf{r}_i\otimes \mathbf{Z}_i^T\mathbf{r}_i\right)
\end{align}

In [19]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # TODO: fill m.β, m.L, m.σ² by LS estimates
    # sleep(1e-3) # pretend this takes 1ms
    n = length(m.data)
    
    fill!(m.xty, 0.0)
    fill!(m.ztr, 0.0)
    fill!(m.ztr2, 0.0)
    fill!(m.xtx, 0.0)
    fill!(m.ztz2, 0.0)
    fill!(m.β, 0.0)
    fill!(m.L, 0.0)
    fill!(m.σ², 0.0)
    sum_n = 0
    
    for i in 1:n
        obs = m.data[i]
        
        # β
        axpy!(T(1), obs.xty, m.xty)
        axpy!(T(1), obs.xtx, m.xtx)
        m.σ²[1] += obs.yty
    end
    mul!(m.β, m.xtx \ I, m.xty)
    m.σ²[1] += dot(m.β, BLAS.gemv!('N', T(1), m.xtx, m.β, T(-2), m.xty))
    
    for i in 1:n
        obs = m.data[i]
        
        # σ²
        #copy!(m.xty, obs.xty)
        #m.σ²[1] += obs.yty + dot(m.β, BLAS.gemv!('N', T(1), obs.xtx, m.β, T(-2), m.xty))
        sum_n += size(obs.y, 1)
        
        # 𝚺
        ## Ztr \otimes Ztr
        copy!(m.ztr, obs.zty)
        BLAS.gemv!('N', T(1), obs.ztx, m.β, T(-1), m.ztr)
        axpy!(T(1), kron(m.ztr, m.ztr), m.ztr2)
        ## ZtZ \otimes ZtZ
        axpy!(T(1), kron(obs.ztz, obs.ztz), m.ztz2)
    end
    
    rdiv!(m.σ², sum_n)
    LAPACK.posv!('L', m.ztz2, m.ztr2)
    m.Σ .= reshape(m.ztr2, (q, q))
    m.L .= Matrix(cholesky(Symmetric(m.Σ)).L)
    
    m
end

init_ls!

In [20]:
init_ls!(lmm)
@show logl!(lmm)
@show lmm.β
@show lmm.σ²
@show lmm.L;

logl!(lmm) = -3.3626049341044296e6
lmm.β = [0.18207934611476329, 6.500480700993721, -3.4979107842091595, 1.001113296229795, 5.0002519857919285]
lmm.σ² = [5.7090047334136225]
lmm.L = [1.4069222734993236 0.0 0.0; 0.05159105901325529 1.131979211870369 0.0; 0.04063584138912113 -0.06994463586493148 0.9718256360134827]


### Correctness

Your start points should have a log-likelihood larger than -3.3627e6 (10 pts). The points you get are

In [21]:
# this is the points you get
(logl!(lmm) >  -3.3627e6) * 10

10

### Efficiency

The start point should be computed quickly. Otherwise there is no point using it as a starting point. My median run time is 175μs. You get full credit (10 pts) if the median run time is within 1ms.

In [22]:
bm_init = @benchmark init_ls!($lmm)

BenchmarkTools.Trial: 9195 samples with 1 evaluation.
 Range (min … max):  405.125 μs … 46.213 ms  ┊ GC (min … max):  0.00% … 98.50%
 Time  (median):     450.584 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   542.095 μs ±  1.312 ms  ┊ GC (mean ± σ):  15.77% ±  6.76%

  ▆▇▄▃▂▁▁▁▇██▇▆▄▃▂▁▁ ▁                          ▁▁▁▂▁       ▁  ▂
  ████████████████████▇▆▆▆▆▅▆▅▆▅▆▆▅▁▆▇▇▆▆▄▆▅▅▅▆█████████▇▇▇███ █
  405 μs        Histogram: log(frequency) by time       684 μs <

 Memory estimate: 1.09 MiB, allocs estimate: 8010.

In [23]:
# this is the points you get
clamp(1 / (median(bm_init).time / 1e6) * 10, 0, 10)

10.0

## Q6. NLP via MathOptInterface.jl

We define the NLP problem using the modelling tool [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl). Start-up code is given below. Modify if necessary to accomodate your own code.

In [24]:
"""
    fit!(m::LmmModel, solver=Ipopt.Optimizer())

Fit an `LmmModel` object by MLE using a nonlinear programming solver. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m :: LmmModel{T},
        solver = Ipopt.Optimizer()
    ) where T <: AbstractFloat
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    npar = p + ((q * (q + 1)) >> 1) + 1
    # prep the MOI
    MOI.empty!(solver)
    # set lower bounds and upper bounds of parameters
    # q diagonal entries of Cholesky factor L should be >= 0
    # σ² should be >= 0
    lb   = fill(0.0, q + 1)
    ub   = fill(Inf, q + 1)
    NLPBlock = MOI.NLPBlockData(MOI.NLPBoundsPair.(lb, ub), m, true)
    MOI.set(solver, MOI.NLPBlock(), NLPBlock)
    # start point
    params = MOI.add_variables(solver, npar)    
    par0   = Vector{T}(undef, npar)
    modelpar_to_optimpar!(par0, m)    
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), params[i], par0[i])
    end
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MAX_SENSE)
    # optimize
    MOI.optimize!(solver)
    optstat = MOI.get(solver, MOI.TerminationStatus())
    optstat in (MOI.LOCALLY_SOLVED, MOI.ALMOST_LOCALLY_SOLVED) || 
        @warn("Optimization unsuccesful; got $optstat")
    # update parameters and refresh gradient
    xsol = [MOI.get(solver, MOI.VariablePrimal(), MOI.VariableIndex(i)) for i in 1:npar]
    optimpar_to_modelpar!(m, xsol)
    logl!(m, true)
    m
end

"""
    ◺(n::Integer)

Triangular number `n * (n + 1) / 2`.
"""
@inline ◺(n::Integer) = (n * (n + 1)) >> 1

"""
    modelpar_to_optimpar!(par, m)

Translate model parameters in `m` to optimization variables in `par`.
"""
function modelpar_to_optimpar!(
        par :: Vector,
        m   :: LmmModel
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(par, m.β)
    # L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        par[offset] = m.L[i, j]
        offset += 1
    end
    # σ²
    par[end] = m.σ²[1]
    par
end

"""
    optimpar_to_modelpar!(m, par)

Translate optimization variables in `par` to the model parameters in `m`.
"""
function optimpar_to_modelpar!(
        m   :: LmmModel, 
        par :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(m.β, 1, par, 1, p)
    # L
    fill!(m.L, 0)
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        m.L[i, j] = par[offset]
        offset   += 1
    end
    # σ²
    m.σ²[1] = par[end]    
    m
end

function MOI.initialize(
        m                  :: LmmModel, 
        requested_features :: Vector{Symbol}
    )
    for feat in requested_features
        if !(feat in MOI.features_available(m))
            error("Unsupported feature $feat")
        end
    end
end

MOI.features_available(m::LmmModel) = [:Grad, :Hess, :Jac]

function MOI.eval_objective(
        m   :: LmmModel, 
        par :: Vector
    )
    optimpar_to_modelpar!(m, par)
    logl!(m, false) # don't need gradient here
end

function MOI.eval_objective_gradient(
        m    :: LmmModel, 
        grad :: Vector, 
        par  :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    optimpar_to_modelpar!(m, par) 
    obj = logl!(m, true)
    # gradient wrt β
    copyto!(grad, m.∇β)
    # gradient wrt L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        grad[offset] = m.∇L[i, j]
        offset += 1
    end
    # gradient with respect to σ²
    grad[end] = m.∇σ²[1]
    # return objective
    obj
end

function MOI.eval_constraint(m::LmmModel, g, par)
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    gidx   = 1
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        if i == j
            g[gidx] = par[offset]
            gidx   += 1
        end
        offset += 1
    end
    g[end] = par[end]
    g
end

function MOI.jacobian_structure(m::LmmModel)
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    row  = collect(1:(q + 1))
    col  = Int[]
    offset = p + 1
    for j in 1:q, i in j:q
        (i == j) && push!(col, offset)
        offset += 1
    end
    push!(col, offset)
    [(row[i], col[i]) for i in 1:length(row)]
end

MOI.eval_constraint_jacobian(m::LmmModel, J, par) = fill!(J, 1)

function MOI.hessian_lagrangian_structure(m::LmmModel)
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)    
    q◺   = ◺(q)
    # we work on the upper triangular part of the Hessian
    arr1 = Vector{Int}(undef, ◺(p) + ◺(q◺) + q◺ + 1)
    arr2 = Vector{Int}(undef, ◺(p) + ◺(q◺) + q◺ + 1)
    # Hββ block
    idx  = 1    
    for j in 1:p, i in 1:j
        arr1[idx] = i
        arr2[idx] = j
        idx      += 1
    end
    # HLL block
    for j in 1:q◺, i in 1:j
        arr1[idx] = p + i
        arr2[idx] = p + j
        idx      += 1
    end
    # HLσ² block
    for i in (p + 1):(p + q◺)
        arr1[idx] = i
        arr2[idx] = p + q◺ + 1
        idx      += 1
    end
    # Hσ²σ² block
    arr1[idx] = p + q◺ + 1
    arr2[idx] = p + q◺ + 1
    [(arr1[i], arr2[i]) for i in 1:length(arr1)]
end

function MOI.eval_hessian_lagrangian(
        m   :: LmmModel, 
        H   :: AbstractVector{T},
        par :: AbstractVector{T}, 
        σ   :: T, 
        μ   :: AbstractVector{T}
    ) where {T}    
    p  = size(m.data[1].X, 2)
    q  = size(m.data[1].Z, 2)    
    q◺ = ◺(q)
    optimpar_to_modelpar!(m, par)
    logl!(m, true, true)
    # Hββ block
    idx = 1    
    @inbounds for j in 1:p, i in 1:j
        H[idx] = m.Hββ[i, j]
        idx   += 1
    end
    # HLL block
    @inbounds for j in 1:q◺, i in 1:j
        H[idx] = m.HLL[i, j]
        idx   += 1
    end
    # HLσ² block
    @inbounds for j in 1:q, i in j:q
        H[idx] = m.Hσ²L[i, j]
        idx   += 1
    end
    # Hσ²σ² block
    H[idx] = m.Hσ²σ²[1, 1]
    lmul!(σ, H)
end

## Q7. (20 pts) Test drive

Now we can run any NLP solver (supported by MathOptInterface.jl) to compute the MLE. For grading purpose, let's use the `:LD_MMA` ([Method of Moving Asymptotes](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/#mma-method-of-moving-asymptotes-and-ccsa)) algorithm in NLopt.jl.

In [25]:
# initialize from least squares
init_ls!(lmm)
println("objective value at starting point: ", logl!(lmm)); println()

# NLopt (LD_MMA) obj. val = -2.8400587866501966e6
NLopt_solver = NLopt.Optimizer()
MOI.set(NLopt_solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
@time fit!(lmm, NLopt_solver)

println("objective value at solution: $(logl!(lmm)))")
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)
println("gradient @ solution:")
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L
@show norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²])

objective value at starting point: -3.3626049341044296e6

  0.682982 seconds (2.35 M allocations: 117.570 MiB, 86.17% compilation time)
objective value at solution: -2.8400587866501906e6)
solution values:
lmm.β = [0.18147761275318702, 6.500383554079115, -3.499864289622533, 0.9997119258250217, 4.9992294818666485]
lmm.σ² = [1.4987345756252395]
lmm.L * transpose(lmm.L) = [1.98362662028152 0.06578009192551917 0.0551714489185263; 0.06578009192551917 1.2814411901189633 -0.09059652721670342; 0.0551714489185263 -0.09059652721670342 0.9434277102704337]
gradient @ solution:
lmm.∇β = [0.020713646000421804, -0.007673049072174365, -0.007696684597899406, -0.0006715494805469291, -0.0007028968814868364]
lmm.∇σ² = [-0.009189199374304735]
lmm.∇L = [-0.0001604167818254836 -0.002991795288477183 0.053539153576452565; 0.001976753047221549 0.00021100492663237756 0.0013633095754418173; 0.07847071933983868 0.0006940645047697002 0.010655693226430092]
norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²]) = 0.083

0.08311566230851339

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [26]:
# objective at solution should be close enough to the optimal
@assert logl!(lmm) > -2.840059e6
# gradient at solution should be small enough
@assert norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²]) < 0.1

### Efficiency

My median run time is 50ms. You get 10 points if your median time is within 1s(=1000ms).

In [27]:
NLopt_solver = NLopt.Optimizer()
MOI.set(NLopt_solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
bm_mma = @benchmark fit!($lmm, $(NLopt_solver)) setup=(init_ls!(lmm))

BenchmarkTools.Trial: 55 samples with 1 evaluation.
 Range (min … max):  91.441 ms …  92.414 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     91.941 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   91.953 ms ± 253.821 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █  ▁    ▁▁ ▁▁▁  ▁▁▁▁██▁ ██ ███▁█▁ ▁█▁██ █ ▁ ▁ ▁▁▁   █▁▁▁▁  █  
  █▁▁█▁▁▁▁██▁███▁▁███████▁██▁██████▁█████▁█▁█▁█▁███▁▁▁█████▁▁█ ▁
  91.4 ms         Histogram: frequency by time         92.4 ms <

 Memory estimate: 18.18 KiB, allocs estimate: 523.

In [28]:
# this is the points you get
clamp(1 / (median(bm_mma).time / 1e9) * 10, 0, 10)

10.0

## Q8. (10 pts) Gradient free vs gradient-based methods

Advantage of using a modelling tool such as MathOptInterface.jl is that we can easily switch the backend solvers. For a research problem, we never know beforehand which solver works best. 

Try different solvers in the NLopt.jl and Ipopt.jl packages. Compare the results in terms of run times (the shorter the better), objective values at solution (the larger the better), and gradients at solution (closer to 0 the better). Summarize what you find.

See this [page](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/) for the descriptions of algorithms in NLopt.

Documentation for the Ipopt can be found [here](https://coin-or.github.io/Ipopt/).  

In [29]:
# vector of solvers to compare
solvers = ["NLopt (LN_COBYLA, gradient free)", "NLopt (LD_MMA, gradient-based)", 
    "Ipopt (L-BFGS)"]

function setup_solver(s::String)
    if s == "NLopt (LN_COBYLA, gradient free)"
        solver = NLopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("algorithm"), :LN_COBYLA)
    elseif s == "NLopt (LD_MMA, gradient-based)"
        solver = NLopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
    elseif s == "Ipopt (L-BFGS)"
        solver = Ipopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("print_level"), 0)
        MOI.set(solver, MOI.RawOptimizerAttribute("hessian_approximation"), "limited-memory")
        MOI.set(solver, MOI.RawOptimizerAttribute("tol"), 1e-6)
    elseif s == "Ipopt (use FIM)"
        # Ipopt (use Hessian) obj val = -2.8400587866468e6
        solver = Ipopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("print_level"), 0)        
    else
        error("unrecognized solver $s")
    end
    solver
end

# containers for results
runtime = zeros(length(solvers))
objvals = zeros(length(solvers))
gradnrm = zeros(length(solvers))

for i in 1:length(solvers)
    solver = setup_solver(solvers[i])
    bm = @benchmark fit!($lmm, $solver) setup = (init_ls!(lmm))
    runtime[i] = median(bm).time / 1e9
    objvals[i] = logl!(lmm, true)
    gradnrm[i] = norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²])
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [30]:
# display results
pretty_table(
    hcat(solvers, runtime, objvals, gradnrm),
    header = ["Solver", "Runtime", "Log-Like", "Gradiant Norm"],
    formatters = (ft_printf("%5.2f", 2), ft_printf("%8.8f", 3:4))
    )

┌──────────────────────────────────┬─────────┬───────────────────┬───────────────┐
│                           Solver │ Runtime │          Log-Like │ Gradiant Norm │
├──────────────────────────────────┼─────────┼───────────────────┼───────────────┤
│ NLopt (LN_COBYLA, gradient free) │    0.45 │ -2840081.03741792 │  839.00644376 │
│   NLopt (LD_MMA, gradient-based) │    0.09 │ -2840058.78665019 │    0.08311566 │
│                   Ipopt (L-BFGS) │    1.39 │ -2840058.78664680 │    0.00139067 │
└──────────────────────────────────┴─────────┴───────────────────┴───────────────┘


## Q9. (10 pts) Compare with existing art

Let's compare our method with lme4 package in R and MixedModels.jl package in Julia. Both lme4 and MixedModels.jl are developed mainly by Doug Bates. Summarize what you find.

In [31]:
method  = ["My method", "lme4", "MixedModels.jl"]
runtime = zeros(3)  # record the run times
loglike = zeros(3); # record the log-likelihood at MLE

### Your approach

In [32]:
solver = setup_solver("NLopt (LD_MMA, gradient-based)")
bm_257 = @benchmark fit!($lmm, $solver) setup=(init_ls!(lmm))
runtime[1] = (median(bm_257).time) / 1e9
loglike[1] = logl!(lmm)

-2.8400587866501906e6

### lme4 

In [33]:
R"""
library(lme4)
library(readr)
library(magrittr)

testdata <- read_csv("lmm_data.csv")
"""

┌ Warning: RCall.jl: Loading required package: Matrix
└ @ RCall ~/.julia/packages/RCall/LWzAQ/src/io.jl:172
┌ Warning: RCall.jl: Rows: 1744977 Columns: 8
│ ── Column specification ────────────────────────────────────────────────────────
│ Delimiter: ","
│ dbl (8): ID, Y, X1, X2, X3, X4, Z1, Z2
│ 
│ ℹ Use `spec()` to retrieve the full column specification for this data.
│ ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
└ @ RCall ~/.julia/packages/RCall/LWzAQ/src/io.jl:172


RObject{VecSxp}
# A tibble: 1,744,977 × 8
      ID        Y     X1      X2     X3      X4      Z1      Z2
   <dbl>    <dbl>  <dbl>   <dbl>  <dbl>   <dbl>   <dbl>   <dbl>
 1     1   9.52    0.202 -0.463   0.798  0.734   0.685  -0.570 
 2     1  24.4     1.59  -1.95    1.20   1.43    1.64    0.369 
 3     1  -1.99    0.378 -0.0367  1.63  -1.15   -0.818   2.83  
 4     1 -17.4    -1.88   0.375  -0.498 -0.253   1.56    1.68  
 5     1  -0.0704  0.658 -0.165   0.780 -1.23   -0.0288 -1.09  
 6     1  -0.853   0.458 -0.313  -0.512 -0.800  -0.331   1.98  
 7     1  -1.80    0.220  0.328   1.32  -1.01   -0.363  -0.0703
 8     1   5.88    1.30   0.889  -0.854  0.0714 -0.658  -0.0339
 9     1  -9.21   -1.43  -0.522  -0.119 -0.580  -0.155  -1.89  
10     1 -11.3    -0.468 -0.700   0.872 -1.82    1.80    0.492 
# ℹ 1,744,967 more rows


In [34]:
R"""
rtime <- system.time(mmod <- 
  lmer(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID), testdata, REML = FALSE))
"""

RObject{RealSxp}
   user  system elapsed 
 84.058   8.406  92.470 


In [35]:
R"""
rtime <- rtime["elapsed"]
summary(mmod)
rlogl <- logLik(mmod)
"""
runtime[2] = @rget rtime
loglike[2] = @rget rlogl;

### MixedModels.jl


In [44]:
using DataFrames
testdata = CSV.File("lmm_data.csv", types = Dict(1=>String)) |> DataFrame

Row,ID,Y,X1,X2,X3,X4,Z1,Z2
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,9.52102,0.201821,-0.463234,0.797731,0.73357,0.685476,-0.569622
2,1,24.4063,1.58557,-1.94608,1.19787,1.43149,1.63962,0.369053
3,1,-1.99215,0.378332,-0.0367002,1.63072,-1.15031,-0.817843,2.83422
4,1,-17.4233,-1.8826,0.374561,-0.49786,-0.253248,1.56433,1.67857
5,1,-0.0704245,0.658283,-0.165487,0.77951,-1.22763,-0.0287779,-1.09172
6,1,-0.853357,0.457784,-0.313387,-0.512299,-0.800278,-0.330632,1.97609
7,1,-1.80061,0.220461,0.327879,1.32209,-1.01336,-0.362947,-0.0703055
8,1,5.88119,1.30135,0.88884,-0.853941,0.0714372,-0.658202,-0.0338648
9,1,-9.20504,-1.43248,-0.521638,-0.119287,-0.579596,-0.154869,-1.88707


In [45]:
mj = fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
bm_mm = @benchmark fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), $testdata)
loglike[3] = loglikelihood(mj)
runtime[3] = median(bm_mm).time / 1e9

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter ~/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Minimizing 189 	 Time: 0:00:00 ( 0.55 ms/it)


0.6062183545

In [46]:
display(bm_mm)
mj

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  566.675 ms … 769.682 ms  ┊ GC (min … max): 1.01% … 22.94%
 Time  (median):     606.218 ms               ┊ GC (median):    3.24%
 Time  (mean ± σ):   629.265 ms ±  64.796 ms  ┊ GC (mean ± σ):  5.86% ±  7.56%

  █    █  █  ██      █           █                            █  
  █▁▁▁▁█▁▁█▁▁██▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  567 ms           Histogram: frequency by time          770 ms <

 Memory estimate: 841.87 MiB, allocs estimate: 7895.

|             |    Est. |     SE |        z |      p |   σ_ID |
|:----------- | -------:| ------:| --------:| ------:| ------:|
| (Intercept) |  0.1815 | 0.0445 |     4.08 | <1e-04 | 1.4086 |
| X1          |  6.5004 | 0.0009 |  7000.80 | <1e-99 |        |
| X2          | -3.4999 | 0.0009 | -3768.92 | <1e-99 |        |
| X3          |  0.9997 | 0.0009 |  1078.16 | <1e-99 |        |
| X4          |  4.9992 | 0.0009 |  5391.08 | <1e-99 |        |
| Z2          |         |        |          |        | 0.9711 |
| Z1          |         |        |          |        | 1.1319 |
| Residual    |  1.2242 |        |          |        |        |


### Summary

In [47]:
pretty_table(
    hcat(method, runtime, loglike),
    header = ["Method", "Runtime", "Log-Like"],
    formatters = (ft_printf("%5.2f", 2), ft_printf("%8.6f", 3))
    )

┌────────────────┬─────────┬─────────────────┐
│         Method │ Runtime │        Log-Like │
├────────────────┼─────────┼─────────────────┤
│      My method │    0.09 │ -2840058.786650 │
│           lme4 │   92.47 │ -2840058.786650 │
│ MixedModels.jl │    0.61 │ -2840058.786743 │
└────────────────┴─────────┴─────────────────┘


## Q9. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on complex longitudinal data sets with millions of records. And you beat current software by XXX fold.